In [815]:
from matplotlib import pyplot as plt
import pyodbc
import pandas as pd
import numpy as np
import json
import folium

import warnings
warnings.filterwarnings(action='ignore')

from branca.element import Template, MacroElement

## 데이터 로드

In [816]:
cnxn = pyodbc.connect("DSN=DSN; uid=scott; pwd=tiger")

In [817]:
query = '''SELECT 
    ADDRESS, ROUND(AVG(CREDIT_LIMIT),2) AS AVG_CREDIT_LIMIT, ROUND(AVG(AGE),2) AS AVG_AGE,
    count(GENDER) AS COUNT_ALL,
    CONCAT(TO_CHAR(ROUND(count(case when GENDER = 'F' then 1 END)/count(GENDER),4)*100),'%') AS COUNT_FEMALE,
    CONCAT(TO_CHAR(ROUND(count(case when GENDER = 'M' then 1 END)/count(GENDER),4)*100),'%') AS COUNT_MALE,
    CASE
        WHEN COUNT(GENDER)<=20000 THEN '회원 수 적음'
        WHEN COUNT(GENDER)<=50000 THEN '회원 수 보통'
        WHEN COUNT(GENDER)>50000 THEN '회원 수 많음'
    END AS COUNT_STAT,
    CASE
        WHEN COUNT(GENDER)<=20000 THEN 'red'
        WHEN COUNT(GENDER)<=50000 THEN 'orange'
        WHEN COUNT(GENDER)>50000 THEN 'green'
    END AS COUNT_COLOR
FROM(
SELECT 
    REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(RTRIM(SUBSTR(ADDRESS1, 0, INSTR(ADDRESS1, ' ', 1, 2))),
    'uC778천 강화군', '인천 강화군'),
    '강원 홍천uAD70', '강원 홍천군'),
    '강원 삼uCC99시', '강원 삼척시'), 
    '충북 uCDA9주시','충북 충주시'),
    '경기 안uC591시','경기 안양시')
    AS ADDRESS,
    CREDIT_LIMIT,
    TRUNC((SYSDATE - BIRTH_DT) / 365) AS AGE,
    GENDER
    FROM CUSTOMER
)
GROUP BY ADDRESS
ORDER BY ADDRESS;'''

In [818]:
df = pd.read_sql(query, cnxn)

In [819]:
df.head()

,ADDRESS,AVG_CREDIT_LIMIT,AVG_AGE,COUNT_ALL,COUNT_FEMALE,COUNT_MALE,COUNT_STAT,COUNT_COLOR
0,강원 강릉시,2497.27,42.08,33171.0,52.12%,47.88%,회원 수 보통,orange
1,강원 고성군,2483.74,42.00,12128.0,51.68%,48.32%,회원 수 적음,red
2,강원 동해시,2494.28,42.03,14226.0,51.83%,48.17%,회원 수 적음,red
3,강원 삼척시,2500.71,41.89,21882.0,52.5%,47.5%,회원 수 보통,orange
4,강원 속초시,2491.17,41.94,6575.0,52.03%,47.97%,회원 수 적음,red


## json load

In [820]:
file_path = "./sigungu.json"

In [821]:
with open('sigungu.json', encoding='UTF-8') as json_file:
    json_data = json.load(json_file)

In [822]:
location=pd.DataFrame(json_data).transpose()

In [823]:
location.reset_index(inplace=True)

In [824]:
location['ADDRESS']=''
for i in range(len(location)):
    location['ADDRESS'][i] = location['index'][i].replace("서울특별시/", "서울 ").replace("부산광역시/", "부산 "). replace("대구광역시/", "대구 "). replace("인천광역시/", "인천 "). replace("광주광역시/", "광주 "). replace("대전광역시/", "대전 "). replace("울산광역시/", "울산 "). replace("경기도/", "경기 "). replace("강원도/", "강원 "). replace("충청북도/", "충북 "). replace("충청남도/", "충남 "). replace("전라북도/", "전북 "). replace("전라남도/", "전남 "). replace("경상북도/", "경북 "). replace("경상남도/", "경남 "). replace("울산광역시/", "울산 ").replace('제주특별자치도/',"제주 ")

In [825]:
df_loc=pd.merge(df,location[['lat','long','ADDRESS']], how='right', on='ADDRESS')

In [826]:
df_loc

,ADDRESS,AVG_CREDIT_LIMIT,AVG_AGE,COUNT_ALL,COUNT_FEMALE,COUNT_MALE,COUNT_STAT,COUNT_COLOR,lat,long
0,서울 강남구,2499.76,42.01,62409.0,52.48%,47.52%,회원 수 많음,green,37.4951,127.06278
1,서울 강동구,2510.94,42.01,38118.0,52.57%,47.43%,회원 수 보통,orange,37.55274,127.14546
2,서울 강북구,2494.75,42.03,32945.0,51.89%,48.11%,회원 수 보통,orange,37.6349,127.02015
3,서울 강서구,2482.98,42.08,34307.0,52.52%,47.48%,회원 수 보통,orange,37.56227,126.81622
4,서울 관악구,2506.98,41.91,51465.0,52.3%,47.7%,회원 수 많음,green,37.47876,126.95235
...,...,...,...,...,...,...,...,...,...,...
223,전남 진도군,2516.04,41.81,13145.0,51.73%,48.27%,회원 수 적음,red,34.41018,126.1688
224,전남 곡성군,2490.89,41.80,15402.0,52.32%,47.68%,회원 수 적음,red,35.21449,127.2628
225,전남 구례군,2486.18,42.00,8640.0,53.22%,46.78%,회원 수 적음,red,35.20944,127.46444
226,제주 제주시,2513.96,42.00,34334.0,53.28%,46.72%,회원 수 보통,orange,33.50972,126.52194


In [827]:
df_loc=df_loc.dropna(axis=0).reset_index()

## 버블 차트

In [828]:
cust_map = folium.Map(
    location=[37.413294,127.0016985],
    zoom_start=10)
marker_cluster = MarkerCluster().add_to(cust_map)

In [829]:
for i in range(len(df_loc)):
    folium.Circle([df_loc['lat'][i],df_loc['long'][i]],
                  radius=df_loc['COUNT_ALL'][i]/10,
                  color='crimson',
                  fill=True,
                  fill_color='crimson',
                  popup=folium.Popup('<strong>' + df_loc['ADDRESS'][i]+'</strong><br>'+
                                       '<strong>''회원수 : ''</strong>'+ str(df_loc['COUNT_ALL'][i])+'<br>'+
                                       '<strong>''범위 : ''</strong>' + df_loc['COUNT_STAT'][i]+'<br>'+
                                       '<strong>''평균나이 : ''</strong>' + str(df_loc['AVG_AGE'][i])+'<br>'+
                                       '<strong>''평균신용한도 : ''</strong>'+ str(round(df_loc['AVG_CREDIT_LIMIT'][i],2)) +'<br>'+
                                       '<strong>''남성회원비율 : ''</strong>' + df_loc['COUNT_MALE'][i]+'<br>'+
                                       '<strong>''여성회원비율 : ''</strong>' + df_loc['COUNT_FEMALE'][i]+'<br>'
                                       , max_width=500)).add_to(cust_map)

In [830]:
cust_map

In [840]:
cust_map.save('py_map1.html')

## 마커 지도

In [846]:
cust_map2 = folium.Map(
    location=[37.413294,127.0016985],
    zoom_start=10)
marker_cluster = MarkerCluster().add_to(cust_map2)

In [847]:
for i in range(len(df_loc)):
    folium.Marker([df_loc['lat'][i],df_loc['long'][i]],
                    icon=folium.Icon(color=df_loc['COUNT_COLOR'][i]),
                    popup=folium.Popup('<strong>' + df_loc['ADDRESS'][i]+'</strong><br>'+
                                       '<strong>''회원수 : ''</strong>'+ str(df_loc['COUNT_ALL'][i])+'<br>'+
                                       '<strong>''범위 : ''</strong>' + df_loc['COUNT_STAT'][i]+'<br>'+
                                       '<strong>''평균나이 : ''</strong>' + str(df_loc['AVG_AGE'][i])+'<br>'+
                                       '<strong>''평균신용한도 : ''</strong>'+ str(round(df_loc['AVG_CREDIT_LIMIT'][i],2)) +'<br>'+
                                       '<strong>''남성회원비율 : ''</strong>' + df_loc['COUNT_MALE'][i]+'<br>'+
                                       '<strong>''여성회원비율 : ''</strong>' + df_loc['COUNT_FEMALE'][i]+'<br>'
                                       , max_width=500)).add_to(cust_map2)

In [848]:
template = """
{% macro html(this, kwargs) %}

<!doctype html>
<html lang="en">
<head>
  <meta charset="utf-8">
  <meta name="viewport" content="width=device-width, initial-scale=1">
  <title>각 지역별 회원 정보</title>
  <link rel="stylesheet" href="//code.jquery.com/ui/1.12.1/themes/base/jquery-ui.css">

  <script src="https://code.jquery.com/jquery-1.12.4.js"></script>
  <script src="https://code.jquery.com/ui/1.12.1/jquery-ui.js"></script>
  
  <script>
  $( function() {
    $( "#maplegend" ).draggable({
                    start: function (event, ui) {
                        $(this).css({
                            right: "auto",
                            top: "auto",
                            bottom: "auto"
                        });
                    }
                });
});

  </script>
</head>
<body>

 
<div id='maplegend' class='maplegend' 
    style='position: absolute; z-index:9999; border:2px solid grey; background-color:rgba(255, 255, 255, 0.8);
     border-radius:6px; padding: 10px; font-size:14px; right: 20px; bottom: 20px;'>
     
<div class='legend-title'>회원 수 범위</div>
<div class='legend-scale'>
  <ul class='legend-labels'>
    <li><span style='background:green;opacity:0.7;'></span>5만명 이상</li>
    <li><span style='background:orange;opacity:0.7;'></span>2만명 이상 5만명 미만</li>
    <li><span style='background:red;opacity:0.7;'></span>2만명 이하</li>
    

  </ul>
</div>
</div>
 
</body>
</html>

<style type='text/css'>
  .maplegend .legend-title {
    text-align: left;
    margin-bottom: 5px;
    font-weight: bold;
    font-size: 90%;
    }
  .maplegend .legend-scale ul {
    margin: 0;
    margin-bottom: 5px;
    padding: 0;
    float: left;
    list-style: none;
    }
  .maplegend .legend-scale ul li {
    font-size: 80%;
    list-style: none;
    margin-left: 0;
    line-height: 18px;
    margin-bottom: 2px;
    }
  .maplegend ul.legend-labels li span {
    display: block;
    float: left;
    height: 16px;
    width: 30px;
    margin-right: 5px;
    margin-left: 0;
    border: 1px solid #999;
    }
  .maplegend .legend-source {
    font-size: 80%;
    color: #777;
    clear: both;
    }
  .maplegend a {
    color: #777;
    }
</style>
{% endmacro %}"""

macro = MacroElement()
macro._template = Template(template)

cust_map2.get_root().add_child(macro)

In [842]:
cust_map2

In [849]:
cust_map2.save('py_map2.html')